# Практическое занятие №2 - локализация объектов на изображениях

1. Разметка данных с помощью виджета QSL для Jupyter (https://github.com/faustomorales/qsl)
2. Расширение выборки за счет модификации размеченных данных (процедура аугоментация).
3. Знакомство с архитектурой нейронных сетей для локализации объектов на изображении.
4. Обучение нейронных сетей для детекции объектов.
5. Проверка обученной модели для обработки видео потока.
6. Трекинг объектов.
7. Распознавание сцен с помощью простого классификатора.
9. Разработка простой системы управления по данным с видео-камеры.

In [1]:
!pip install -U -q qsl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.8 MB/s eta 0:00:00


In [2]:
# Активация расширенных виджетов в Colab (можно закомментировать при запуске в Jupyter)
import google.colab
google.colab.output.enable_custom_widget_manager()

In [4]:
import qsl

In [5]:
%%writefile images.csv
target
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_001.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_002.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_003.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_005.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_006.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_007.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_008.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_009.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_010.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_3/image_011.jpg

Writing images.csv


In [6]:
import pandas

image_file = pandas.read_csv('images.csv')

In [7]:
labeler = qsl.MediaLabeler(
    items=image_file.to_dict(orient='records'),
    config={
        "regions": [
            {"name": "Object", "multiple": True, "options": [{"name": "bottle"}, {"name": "bug"}]}
        ]
    })
labeler.mode = "dark"
labeler

MediaLabeler(buttons={'prev': False, 'next': True, 'save': True, 'config': True, 'delete': False, 'ignore': Tr…

In [27]:
with open('result.json', 'w') as file:
    file.write(json.dumps(labeler.items, sort_keys=True, indent=4))

In [ ]:
labels = None
with open('result.json', 'r') as file:
    labels = file.read()
    print(labels)

In [33]:
!wget 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/object_masks.json'

--2023-02-16 14:03:10--  https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/object_masks.json
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/object_masks.json [following]
--2023-02-16 14:03:10--  https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/object_masks.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119085 (116K) [text/plain]
Saving to: ‘object_masks.json’

object_masks.json   100%[===================>] 116.29K  --.-KB/s    in 0.03s   

2023-02-16 14:03:10 (4.34 MB/s) - ‘obj

In [47]:
import json

labels = None
with open('object_masks.json', 'r') as file:
    labels = file.read()
label_data = json.loads(labels) #.replace('True', 'true').replace('False', 'false'))

In [48]:
labeler = qsl.MediaLabeler(
    items=label_data,
    
    #image_file.to_dict(orient='records'),
    config={
        "regions": [
            {"name": "Type", "multiple": True, "options": [{"name": "bottle"}, {"name": "bag"}]}
        ]
    })
labeler.labels#  = labels
labeler.mode = "dark"
labeler

MediaLabeler(buttons={'prev': False, 'next': True, 'save': True, 'config': True, 'delete': True, 'ignore': Fal…

In [55]:
class Struct(object):

    def __init__(self, data):
        for name, value in data.items():
            setattr(self, name, self._wrap(value))

    def _wrap(self, value):
        if isinstance(value, (tuple, list, set, frozenset)): 
            return type(value)([self._wrap(v) for v in value])
        else:
            return Struct(value) if isinstance(value, dict) else value

    def __str__(self):
        result = '{'
        first = True
        for field in self.__dict__.keys():
            if not first:
                result += ', '
            else:
                first = False
            result += f'{field}: {str(self.__dict__[field])}'
        return result + '}'

    def __repr__(self):
        return str(self)

In [56]:
labels = [Struct(label) for label in label_data] # [0]['labels']['polygons'])
labels[0]

{ignore: False, labels: {boxes: [], dimensions: {height: 914, width: 1628}, image: {}, masks: [], polygons: [{labels: {Object: ['bug']}, points: [{x: 0.642587170887295, y: 0.3516193884040677}, {x: 0.6700005413571787, y: 0.3926353214413227}, {x: 0.7643025357735784, y: 0.45122951149454404}, {x: 0.8235154159885271, y: 0.5117768412162061}, {x: 0.8728594828343176, y: 0.5723241709378682}, {x: 0.9046589925793826, y: 0.5977149866275975}, {x: 0.909045131854564, y: 0.6191995229804453}, {x: 0.934265432686857, y: 0.6133401039751232}, {x: 0.9638718727943313, y: 0.6367777799964117}, {x: 0.9265896889552896, y: 0.7012313890549553}, {x: 0.8893075051162478, y: 0.7656849981134988}, {x: 0.8706664131967269, y: 0.8008415121454316}, {x: 0.8114535329817782, y: 0.8027946518138723}, {x: 0.7686886750487598, y: 0.785216394797906}, {x: 0.7423718393976715, y: 0.7754506964557024}, {x: 0.7335995608473087, y: 0.7344347634184474}, {x: 0.7259238171157413, y: 0.7246690650762438}, {x: 0.7204411430217645, y: 0.697325109718

# Аугоментация данны

Подробный обзор стратегий см. в статье
https://arxiv.org/pdf/2205.01491.pdf